# Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Code IRIS (code quartier de l'INSEE)
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import logging
logging.basicConfig(filename='get_gps_iris.log', level=logging.INFO)

with open('get_gps_iris.log', 'w'):
    pass

In [3]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

## Lecture du fichier Adresses

In [20]:
# Chargement du jeu de données
df = pd.read_csv('01A - export_dvf_adresses_valeursfoncieres-2020.txt.csv', sep=",",
                dtype = {'Code postal': str, 'Code departement': str, 'Code commune INSEE': str})
df.head()

,Unnamed: 0,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse
0,0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,3,NaN,NaN,EN COROBERT,1290,LAIZ,01,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
4,6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [5]:
print(df.columns)

Index(['Unnamed: 0', 'No voie', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune INSEE', 'Ville',
       'Adresse'],
      dtype='object')


## Paramètres

In [6]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 0
row_last = df.shape[0]

### Renseigner manuellement si on souhaite ne parser qu'une partie des adresses
# row_first = 0
# row_last = 50

print("Calcul des coordonnées GPS et Codes IRIS de " + str(row_first) + " à " + str(row_last - 1))
logging.info("Calcul des coordonnées GPS et Codes IRIS de " + str(row_first) + " à " + str(row_last - 1))

Calcul des coordonnées GPS et Codes IRIS de 0 à 49


In [7]:
# Fréquence de l'export intermédiaire (si problème d'API)
freq_export = 1000

## Sélection des adresses à parser pour ajout des coordonnées GPS et codes IRIS

In [8]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = df.iloc[row_first:row_last, :]
selec.shape

(50, 10)

In [21]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [10]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [11]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [12]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec = selec.drop(columns = {'Unnamed: 0'})
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,"46.1832170988,5.32007207542"
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,"46.2469919709,4.89933490972"
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,"46.2469919709,4.89933490972"
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,"46.2469919709,4.89933490972"
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,"46.1711903413,5.21802169564"


In [13]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,46.1832170988,5.32007207542
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,46.2469919709,4.89933490972
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,46.2469919709,4.89933490972
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,46.2469919709,4.89933490972
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,46.1711903413,5.21802169564
5,NaN,NaN,CHAMP PORTIER,1370,VAL-REVERMONT,01,01426,1370 VAL REVERMONT,CHAMP PORTIER 1370 VAL-REVERMONT,NaN,NaN,NaN
6,NaN,NaN,CHAMPEL,1270,COLIGNY,01,01108,1270 COLIGNY,CHAMPEL 1270 COLIGNY,01108,46.3866637865,5.3308227014
7,5367.0,NaN,LORETTE,1270,COLIGNY,01,01108,1270 COLIGNY,5367 LORETTE 1270 COLIGNY,01108,46.3866637865,5.3308227014
8,31.0,RUE,COMTE DE LA TEYSSONNIERE,1000,BOURG-EN-BRESSE,01,01053,1000 BOURG EN BRESSE,31 RUE COMTE DE LA TEYSSONNIERE 1000 BOURG-EN-...,01053,46.2051520382,5.24602125501
9,6.0,AV,DU CHAMP DE FOIRE,1000,BOURG-EN-BRESSE,01,01053,1000 BOURG EN BRESSE,6 AV DU CHAMP DE FOIRE 1000 BOURG-EN-BRESSE,01053,46.2051520382,5.24602125501


In [14]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,46.1832170988,5.32007207542,,,
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,46.1711903413,5.21802169564,,,


In [15]:
selec.shape

(50, 15)

## Récupération des codes IRIS et coordonnées GPS à partir de l'API Nominatim et geo.api.gouv

In [16]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="pyPredImmo")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")

logging.info("Heure début : " + time.strftime('%H:%M:%S', time.gmtime(start)) + "\n")

n=0

for row in range(0, row_last - row_first):
    
    iris = ""
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""

    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]

    # Interrogation de l'API Nominatim
    try:
        location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 5)
    
    except requests.exceptions.RequestException as e:
        latitude_api = ""
        longitude_api = ""
        raise SystemExit(e)
        
    if location is None:                      # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            logging.info(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude_api == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS pour code IRIS")
            logging.info(adresse + ": pas de coordonnees GPS pour code IRIS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            try:
                result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
                                 + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            except requests.exceptions.RequestException as e: 
                raise SystemExit(e)
            
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude_api) + "," + str(longitude_api) + " (code IRIS = " + str(iris) + ")")
   
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude_api
    selec['lat'][row] = latitude_api
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        logging.info("Etape : " + str(n) + ", Temps pour 100 items: " + str(time.time() - etape) + " sec")
        etape = time.time()

    if (n % freq_export == 0):
        selec.iloc[(n - freq_export) : n, :].to_csv(
            'export_adresses_inter_' + str(row_first + n - freq_export) + "-" + str(row_first + n - 1)+'.csv')
        logging.info("02 - Export intermédiaire : " + str(row_first + n - freq_export) + "-" + str(row_first + n - 1))

        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

logging.info("Heure fin : " + time.strftime('%H:%M:%S', time.gmtime(end)) + "\n")
logging.info("Temps de calcul : " + str(end - start) + ' sec')

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
15:07:03


FORTUNAT 1250 CEYZERIAT ===> 46.1737528,5.3230074 (code IRIS = 010720000)
EN COROBERT 1290 LAIZ ===> 46.2483684,4.8819132 (code IRIS = 012030000)
RUE DE LA CHARTREUSE 1960 PERONNAS ===> 46.1849787,5.2275831 (code IRIS = 012890102)
CHAMP PORTIER 1370 VAL-REVERMONT ===> 46.2644815,5.3429528 (code IRIS = 014260000)
CHAMPEL 1270 COLIGNY ===> 46.3809924,5.3493734 (code IRIS = 011080000)
5367  LORETTE 1270 COLIGNY ===> 46.3804688,5.3514995 (code IRIS = 011080000)
31 RUE COMTE DE LA TEYSSONNIERE 1000 BOURG-EN-BRESSE ===> 46.197026,5.208966 (code IRIS = 010530203)
6 AV DU CHAMP DE FOIRE 1000 BOURG-EN-BRESSE ===> 46.206965,5.2299233 (code IRIS = 010530101)
LES CADALLES 1000 SAINT-DENIS-LES-BOURG ===> 46.2096756,5.2013121 (code IRIS = 013440000)
LES FOSSES 1440 VIRIAT ===> 46.2391305,5.2289851 (code IRIS = 014510101)
5104  COLIGNY LE BAS 1270 COLIGNY ===> 46.3880458,5.347011 (code IRIS = 011080000)
16 RUE DES GRAVES 1000 BOURG-EN-BRESSE ===> 46.1994951,5.2322911 (code I

In [17]:
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT,01072,46.1832170988,5.32007207542,010720000,5.323007,46.173753
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ,01203,46.2469919709,4.89933490972,,,
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,01203,1290 LAIZ,EN COROBERT 1290 LAIZ,01203,46.2469919709,4.89933490972,012030000,4.881913,46.248368
4,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS,01289,46.1711903413,5.21802169564,012890102,5.227583,46.184979


In [18]:
selec.shape

(50, 15)

## Export de la totalité de la sélection d'adresses après la fin du job

In [19]:
selec.to_csv('02 - export_adresses_' + str(row_first) + "-" + str(row_last - 1)+'.csv', index = False)